### Load Your Libraries

In [65]:
import pandas as pd  # injesting and transforming our raw data
import ast  # abstract syntax tree: convert strings to python object

### load the raw datasets

In [66]:
customers = pd.read_csv("./Data/customers.csv")
offers = pd.read_csv("./Data/offers.csv")
events = pd.read_csv("./Data/events.csv")

### Data Exploration
* size of the data
* data type in the datasets
* find inconsistences in the datasets

In [67]:
def dataset_size(dataframe,table_name):
    size = dataframe.shape
    row = size[0]
    column = size[1]
    print(f"{table_name} has {row} rows and {column} columns.")
    

dataset_size(customers,"customers")
dataset_size(offers,"offers")
dataset_size(events,"events")  


customers has 17000 rows and 5 columns.
offers has 10 rows and 6 columns.
events has 306534 rows and 4 columns.


In [100]:
# customers

print(customers.dtypes)
customers.head()

customer_id                    str
became_member_on    datetime64[us]
gender                         str
age                          int64
income                     float64
dtype: object


,customer_id,became_member_on,gender,age,income
0,68be06ca386d4c31939f3a4f0e3dd783,2017-02-12,unspecified,118,0.0
1,0610b486422d4921ae7d2bf64640c50b,2017-07-15,F,55,112000.0
2,38fe809add3b4fcf9315a9694bb96ff5,2018-07-12,unspecified,118,0.0
3,78afa995795e4d85b5d9ceeca43f5fef,2017-05-09,F,75,100000.0
4,a03223e636434f42ac4c3df47e8bac43,2017-08-04,unspecified,118,0.0


In [73]:
# offers
print(offers.dtypes)
offers.head()

offer_id         str
offer_type       str
difficulty     int64
reward         int64
duration       int64
channels      object
dtype: object


,offer_id,offer_type,difficulty,reward,duration,channels
0,ae264e3637204a6fb9bb56bc8210ddfd,bogo,10,10,7,"[email, mobile, social]"
1,4d5c57ea9a6940dd891ad53e9dbe8da0,bogo,10,10,5,"[web, email, mobile, social]"
2,3f207df678b143eea3cee63160fa8bed,informational,0,0,4,"[web, email, mobile]"
3,9b98b8c7a33c4b65b9aebfe6a799e6d9,bogo,5,5,7,"[web, email, mobile]"
4,0b1e1539f2cc45b7b9fa7c272da2e1d7,discount,20,5,10,"[web, email]"


In [75]:
## Events
print(events.dtypes)
events.head()

customer_id      str
event            str
value            str
time           int64
dtype: object


,customer_id,event,value,time
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0
1,a03223e636434f42ac4c3df47e8bac43,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0
2,e2127556f4f64592b11af22de27a7932,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0
4,68617ca6246f4fbc85e91a2a49552598,offer received,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0


### The Customer Table Transformation
* convert the 'became_memeber_on' column to a date type 
*filling the null values

In [99]:
# convert to date column

customers["became_member_on"] = pd.to_datetime(customers['became_member_on'],format ="%Y%m%d")

In [70]:
## fill in missing values in the gender and income columns
customers['income'] = customers['income'].fillna(0.0)

## Assignment :fill the missing values for gender with 'unspecified'

customers['gender'] = customers['gender'].fillna('unspecified')

### Offers Table Transformation

* transform the channels data to a list object.
* extract the offer_id and channels columns
* explode the channels columns to satisfy the 1NF
* remove the channels columns in the offers tables to satisfy 2NF

In [72]:
offers['channels'] = offers['channels'].apply(ast.literal_eval)

In [ ]:
## Create a junction table

offers_channels = offers [["offer_id","channels"]]
offers_channels

,offer_id,channels
0,ae264e3637204a6fb9bb56bc8210ddfd,"[email, mobile, social]"
1,4d5c57ea9a6940dd891ad53e9dbe8da0,"[web, email, mobile, social]"
2,3f207df678b143eea3cee63160fa8bed,"[web, email, mobile]"
3,9b98b8c7a33c4b65b9aebfe6a799e6d9,"[web, email, mobile]"
4,0b1e1539f2cc45b7b9fa7c272da2e1d7,"[web, email]"
5,2298d6c36e964ae4a3e7e9706d1fb8c2,"[web, email, mobile, social]"
6,fafdcd668e3743c1bb461111dcafc2a4,"[web, email, mobile, social]"
7,5a8bc65990b245e5a138643cd4eb9837,"[email, mobile, social]"
8,f19421c1d4aa40978ebb69ca19b0e20d,"[web, email, mobile, social]"
9,2906b810c7d4411798c6938adc9daaa5,"[web, email, mobile]"


In [78]:
offers_channels = offers_channels.explode('channels')
offers_channels

,offer_id,channels
0,ae264e3637204a6fb9bb56bc8210ddfd,email
0,ae264e3637204a6fb9bb56bc8210ddfd,mobile
0,ae264e3637204a6fb9bb56bc8210ddfd,social
1,4d5c57ea9a6940dd891ad53e9dbe8da0,web
1,4d5c57ea9a6940dd891ad53e9dbe8da0,email
1,4d5c57ea9a6940dd891ad53e9dbe8da0,mobile
1,4d5c57ea9a6940dd891ad53e9dbe8da0,social
2,3f207df678b143eea3cee63160fa8bed,web
2,3f207df678b143eea3cee63160fa8bed,email
2,3f207df678b143eea3cee63160fa8bed,mobile


In [ ]:
offers_transformed = offers.drop(columns=['channels'])
offers_transformed

,offer_id,offer_type,difficulty,reward,duration
0,ae264e3637204a6fb9bb56bc8210ddfd,bogo,10,10,7
1,4d5c57ea9a6940dd891ad53e9dbe8da0,bogo,10,10,5
2,3f207df678b143eea3cee63160fa8bed,informational,0,0,4
3,9b98b8c7a33c4b65b9aebfe6a799e6d9,bogo,5,5,7
4,0b1e1539f2cc45b7b9fa7c272da2e1d7,discount,20,5,10
5,2298d6c36e964ae4a3e7e9706d1fb8c2,discount,7,3,7
6,fafdcd668e3743c1bb461111dcafc2a4,discount,10,2,10
7,5a8bc65990b245e5a138643cd4eb9837,informational,0,0,3
8,f19421c1d4aa40978ebb69ca19b0e20d,bogo,5,5,5
9,2906b810c7d4411798c6938adc9daaa5,discount,10,2,7


### Events Table Transformations

In [82]:
events['value'] = events['value'].apply(ast.literal_eval)

In [88]:
values_info = pd.json_normalize(events['value'])
values_info

,offer id,amount,offer_id,reward
0,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN,NaN,NaN
1,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN,NaN,NaN
2,2906b810c7d4411798c6938adc9daaa5,NaN,NaN,NaN
3,fafdcd668e3743c1bb461111dcafc2a4,NaN,NaN,NaN
4,4d5c57ea9a6940dd891ad53e9dbe8da0,NaN,NaN,NaN
...,...,...,...,...
306529,NaN,1.59,NaN,NaN
306530,NaN,9.53,NaN,NaN
306531,NaN,3.61,NaN,NaN
306532,NaN,3.53,NaN,NaN


In [ ]:
# clean up the  value info 

if 'offer id' in values_info.columns:
    values_info['offer_id'] = values_info['offer_id'].fillna(values_info
    ['offer id'])
    values_info = values_info.drop(columns =['offer id'])
else:
    print('no such columns')  

values_info

,amount,offer_id,reward
0,NaN,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN
1,NaN,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN
2,NaN,2906b810c7d4411798c6938adc9daaa5,NaN
3,NaN,fafdcd668e3743c1bb461111dcafc2a4,NaN
4,NaN,4d5c57ea9a6940dd891ad53e9dbe8da0,NaN
...,...,...,...
306529,1.59,NaN,NaN
306530,9.53,NaN,NaN
306531,3.61,NaN,NaN
306532,3.53,NaN,NaN


In [107]:
events_transformed = events.drop(columns=['value'])
events_transformed = pd.concat([events_transformed,values_info],axis = 1)
events_transformed

,customer_id,event,time,amount,offer_id,reward
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,0,NaN,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN
1,a03223e636434f42ac4c3df47e8bac43,offer received,0,NaN,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN
2,e2127556f4f64592b11af22de27a7932,offer received,0,NaN,2906b810c7d4411798c6938adc9daaa5,NaN
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,0,NaN,fafdcd668e3743c1bb461111dcafc2a4,NaN
4,68617ca6246f4fbc85e91a2a49552598,offer received,0,NaN,4d5c57ea9a6940dd891ad53e9dbe8da0,NaN
...,...,...,...,...,...,...
306529,b3a1272bc9904337b331bf348c3e8c17,transaction,714,1.59,NaN,NaN
306530,68213b08d99a4ae1b0dcb72aebd9aa35,transaction,714,9.53,NaN,NaN
306531,a00058cf10334a308c68e7631c529907,transaction,714,3.61,NaN,NaN
306532,76ddbd6576844afe811f1a3c0fbb5bec,transaction,714,3.53,NaN,NaN


### Assignment
* fill the null values in the amount and the reward columns with 0.0
* convert the time column to datetime object

In [110]:


events_transformed['amount'] = events_transformed['amount'].fillna(0.0)
events_transformed['reward'] = events_transformed['reward'].fillna(0.0)
#events_transformed["time"]= pd.to_datetime(events_transformed['time'],format ="%Y%m%d")



In [111]:
events_transformed

,customer_id,event,time,amount,offer_id,reward
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,0,0.00,9b98b8c7a33c4b65b9aebfe6a799e6d9,0.0
1,a03223e636434f42ac4c3df47e8bac43,offer received,0,0.00,0b1e1539f2cc45b7b9fa7c272da2e1d7,0.0
2,e2127556f4f64592b11af22de27a7932,offer received,0,0.00,2906b810c7d4411798c6938adc9daaa5,0.0
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,0,0.00,fafdcd668e3743c1bb461111dcafc2a4,0.0
4,68617ca6246f4fbc85e91a2a49552598,offer received,0,0.00,4d5c57ea9a6940dd891ad53e9dbe8da0,0.0
...,...,...,...,...,...,...
306529,b3a1272bc9904337b331bf348c3e8c17,transaction,714,1.59,NaN,0.0
306530,68213b08d99a4ae1b0dcb72aebd9aa35,transaction,714,9.53,NaN,0.0
306531,a00058cf10334a308c68e7631c529907,transaction,714,3.61,NaN,0.0
306532,76ddbd6576844afe811f1a3c0fbb5bec,transaction,714,3.53,NaN,0.0
